# Outil de débogage interactif

**Description**

Ce notebook offre un outil interactif (widget) pour visualiser les évènements des matchs répertoriés par saison.


In [70]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import json
import pprint
import glob
from tabulate import tabulate

In [171]:
def plotting_coordinates(info, nb_events, team_away, team_home):
    @interact
    def interact_event(eventIdx = (0, nb_events-1, 1)):
        # Retrieve coord for specified `event` in `game_id`
        coord = info['liveData']['plays']['allPlays'][eventIdx]['coordinates']
        # Retrieve period info 
        period = info['liveData']['plays']['allPlays'][eventIdx]['about']['period']
        # Team's side on rink
        if info['liveData']['linescore']['periods'][period-1]['home']['rinkSide'] == 'right':
            teams_side = team_away+' '*80+team_home
        else:
            teams_side = team_home+' '*80+team_away
        # Plot if coord is not empty
        im = plt.imread('../figures/nhl_rink.png')
        fig, ax = plt.subplots(figsize=(20, 15))
        # Adjust figure size to match rink dimensions
        height_rink, width_rink = 85, 200 # NHL rink dimensions
        ax.imshow(im, extent=[-width_rink/2, width_rink/2, -height_rink/2, height_rink/2])
        if coord:
            ax.scatter(coord['x'], coord['y'], s=300, c='blue')
        plt.xlabel('feet')
        plt.ylabel('feet')
        plt.title(f"{info['liveData']['plays']['allPlays'][eventIdx]['result']['description']}\n{info['liveData']['plays']['allPlays'][eventIdx]['about']['periodTime']} P-{period}\n {teams_side}", fontsize=15)
        plt.show()
        pprint.pprint(info['liveData']['plays']['allPlays'][eventIdx])

def get_match_info(info, gameId=None, round=None, rencontre=None, match=None):
    # Retrieve SoG info
    SoG_home, SoG_away = 0, 0
    for p in info['liveData']['linescore']['periods']:
        SoG_home += p['home']['shotsOnGoal']
        SoG_away += p['away']['shotsOnGoal']
    # Widget with Event ID
    print(info['gameData']['datetime']['dateTime'])
    team_away = info['gameData']['teams']['away']['abbreviation']
    team_home = info['gameData']['teams']['home']['abbreviation']
    if gameId is not None:
        print(f"Game ID: {gameId}; {team_away} (away) vs {team_home} (home) \n")
    else:
        print(f"Playoffs rounds: {round} | rencontre: {rencontre} | match: {match}; {team_away} (away) vs {team_home} (home) \n")
    table = [
        ['', 'Home', 'Away'],
        ['Teams: ', team_home, team_away],
        ['Goals: ', info['liveData']['plays']['currentPlay']['about']['goals']['home'], info['liveData']['plays']['currentPlay']['about']['goals']['away']],
        ['SoG: ', SoG_home, SoG_away],
        ['SO Goals: ', info['liveData']['linescore']['shootoutInfo']['home']['scores'], info['liveData']['linescore']['shootoutInfo']['away']['scores']],
        ['SO Attemps: ', info['liveData']['linescore']['shootoutInfo']['home']['attempts'], info['liveData']['linescore']['shootoutInfo']['away']['attempts']]
    ]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

    return team_away, team_home

In [178]:
@interact
def interactive_debugger(season=['2016_2017', '2017_2018', '2018_2019', '2019_2020', '2020_2021'], season_type=['regular', 'playoff']):
    """
    season: str
        Season to visualize. 
    season_type: str
        Season type to visualize. Can be `reguliere` or `eliminatoire`.
    gameId: int
        Game number to visualize. 0 corresponds to the first game of the season.
    eventIdx: int
        Event number to visualize within the specified game. 0 corresponds to the first event of the game.
    """
    # Get data
    year = season.split("_")[0]

    # Regular season
    if season_type == 'regular':
        season_code = '02'
        nb_games = len(glob.glob(f'../data/raw/{season}/{year+season_code}*'))

        # Widget with Game ID
        @interact
        def interact_game(gameId = (0, nb_games-1, 1)):
            # Load data
            game_number = str(int(gameId)+1).zfill(4)
            info = json.load(open(f'../data/raw/{season}/{year+season_code+game_number}.json'))
            nb_events = len(info['liveData']['plays']['allPlays'])
            # Print match stats overall
            team_away, team_home = get_match_info(info, gameId)
            # Call plotting function
            plotting_coordinates(info, nb_events, team_away, team_home)
    # Playoffs
    else:
        # Widget with Game ID
        @interact
        def interact_game(round = (1, 4, 1)): 
            @interact
            def interact_meeting(rencontre = (1, int(0.5**round*16), 1), match = (1, 7, 1)):
                info = json.load(open(f'../data/raw/{season}/{year+"030"+str(round)+str(rencontre)+str(match)}.json'))
                nb_events = len(info['liveData']['plays']['allPlays'])
                # Print match stats overall
                team_away, team_home = get_match_info(info, round=round, rencontre=rencontre, match=match)
                # Call plotting function
                plotting_coordinates(info, nb_events, team_away, team_home)

interactive(children=(Dropdown(description='season', options=('2016_2017', '2017_2018', '2018_2019', '2019_202…